In [2]:
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import cv2
import random
import pickle
import itertools

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
!pip install scikeras
from scikeras.wrappers import KerasClassifier
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import optimizers
from tensorflow.keras import utils
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn.metrics import confusion_matrix

%matplotlib inline

In [3]:
def plot_results(mod_history, metric, epochs):
      
      # Check out our train loss and test loss over epochs.
      train_metric = mod_history.history[metric]
      val = 'val_' + metric
      test_metric = mod_history.history[val]

      # Set figure size.
      plt.figure(figsize=(12, 8))

      # Generate line plot of training, testing loss over epochs.
      plt.plot(train_metric, label=f'Training {metric}', color='#185fad')
      plt.plot(test_metric, label=f'Testing {metric}', color='orange')

      # Set title
      plt.title(f'Training and Testing {metric} by Epoch', fontsize = 25)
      plt.xlabel('Epoch', fontsize = 18)
      plt.ylabel('Categorical Crossentropy', fontsize = 18)
      plt.xticks(range(0,epochs,5), range(0,epochs,5))
      plt.legend(fontsize = 18);


In [4]:
def make_predictions(mod_name, steps=20):
    preds = mod_name.predict(X_test,steps=steps)
    preds = preds.argmax(axis=-1)

    y_test_labels = np.argmax(y_test, axis=-1)

    cm = confusion_matrix(y_test_labels,preds)

    plot_confusion_matrix(cm, cm_plot_labels, normalize=True,
                          title='Face Shape Normalized')

    plt.show()

In [5]:
cm_plot_labels = ['Heart','Rectangle','Oval','Round', 'Square', 'Triangle']

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.figure(figsize=(16,8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [6]:
def plot_summary_results(mod_name, mod_history, epochs):
    plot_results(mod_history, 'loss',epochs)
    plot_results(mod_history, 'accuracy', epochs)
    make_predictions(mod_name)

In [7]:
path = './pickle_out/rgb/'

X_train = np.asarray(pickle.load(open(path + "pickle_out_rgbX_train_rgb.pickle","rb")))
y_train = np.asarray(pickle.load(open(path + "pickle_out_rgby_train_rgb.pickle","rb")))
X_test = np.asarray(pickle.load(open(path + "pickle_out_rgbX_test_rgb.pickle","rb")))
y_test = np.asarray(pickle.load(open(path + "pickle_out_rgby_test_rgb.pickle","rb")))

In [8]:
print("Data Summary")
print("--------------------")
print(f"X_train shape {X_train.shape}")
print(f"y_train shape {y_train.shape}")
print("--------------------")
print(f"X_test shape {X_test.shape}")
print(f"y_test shape {y_test.shape}")

Data Summary
--------------------
X_train shape (3620, 224, 224, 3)
y_train shape (3620, 6)
--------------------
X_test shape (1204, 224, 224, 3)
y_test shape (1204, 6)


In [17]:
# Path for VGGFace weights

path_vggface = './SavedModels/vgg_face_weights.h5'

In [19]:
# Loading VGG16 as base model

# base_model = VGG16(input_shape=(224, 224, 3),  # same as our input
#                    include_top=False,  # exclude the last layer
#                    weights=path_vggface)  # use VGGFace Weights

base_model = VGG16(input_shape=(224, 224, 3),  # same as our input
                   include_top=False,  # exclude the last layer
                   weights=None)  # use VGGFace Weights

In [20]:
for layer in base_model.layers:
  layer.trainable = False

In [21]:
model_t1 = Sequential()

In [26]:
# Compile and Fit the model

x = layers.Flatten()(base_model.output)

x = layers.Dense(64, activation='relu')(x)  # add 1 fully connected layer, try with 512 first 
x = layers.Dropout(0.5)(x)
x = layers.Dense(6, activation='softmax')(x)  # add final layer

model_t1 = tf.keras.models.Model(base_model.input, x)



model_t1.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])

model_t1.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │     1,605,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,320,774 (62.26 MB)

 Trainable params: 1,606,086 (6.13 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [27]:
datagen = ImageDataGenerator(rotation_range=20, horizontal_flip=True)

In [28]:
datagen.fit(X_train)


In [29]:
history_t1 = model_t1.fit(datagen.flow(X_train, y_train, batch_size=32), 
                          steps_per_epoch=int(len(X_train)/32), epochs=50, 
                          validation_data=(X_test, y_test))

Epoch 1/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 948s 8s/step - accuracy: 0.1763 - loss: 1.7933 - val_accuracy: 0.2284 - val_loss: 1.7896
Epoch 2/50
  1/113 ━━━━━━━━━━━━━━━━━━━━ 11:45 6s/step - accuracy: 0.1562 - loss: 1.7920

/home/vasilemaruseac/jupyter_env/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


113/113 ━━━━━━━━━━━━━━━━━━━━ 257s 2s/step - accuracy: 0.1562 - loss: 1.7920 - val_accuracy: 0.2276 - val_loss: 1.7897
Epoch 3/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 1042s 9s/step - accuracy: 0.1768 - loss: 1.7897 - val_accuracy: 0.1661 - val_loss: 1.7865
Epoch 4/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 240s 2s/step - accuracy: 0.1562 - loss: 1.7970 - val_accuracy: 0.1661 - val_loss: 1.7865
Epoch 5/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 953s 8s/step - accuracy: 0.1853 - loss: 1.7881 - val_accuracy: 0.1728 - val_loss: 1.7834
Epoch 6/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 245s 2s/step - accuracy: 0.0938 - loss: 1.7881 - val_accuracy: 0.1728 - val_loss: 1.7834
Epoch 7/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 1017s 9s/step - accuracy: 0.1968 - loss: 1.7839 - val_accuracy: 0.2849 - val_loss: 1.7811
Epoch 8/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 303s 3s/step - accuracy: 0.1875 - loss: 1.7766 - val_accuracy: 0.2915 - val_loss: 1.7809
Epoch 9/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 1043s 9s/step - accuracy: 0.2135 - loss: 1.7804 - val_accuracy: 0.

KeyboardInterrupt: 

In [ ]:
model_path = './SavedModels/VGGFACE.keras'
tf.keras.models.save_model(
    model_t1, filepath=model_path, overwrite=True, include_optimizer=True, save_format=None)

In [ ]:
filename = model_path + 'vgg16-face-1.keras'   # change the filename for new iterations
model_t1.save(filename)

In [ ]:
loaded_model = tf.keras.models.load_model(filename)
mod_t1_predict = np.argmax(model_t1.predict(X_test), axis=1) 
loaded_t1_predict = np.argmax(loaded_model.predict(X_test), axis=1)

# Check the difference

print(f'Difference in predictions: Saved model vs. original model is {np.sum(loaded_t1_predict - mod_t1_predict)}\nModel was correctly saved.')



In [ ]:
plot_summary_results(model_t1, history_t1, 50)

In [ ]:
history_t2 = model_t1.fit(datagen.flow(X_train, y_train, batch_size=32), 
                          steps_per_epoch=int(len(X_train)/32), epochs=20, 
                          validation_data=(X_test, y_test))

In [ ]:
filename = model_path + 'vgg16-face-2.keras'   # change the filename for new iterations
model_t1.save(filename)

In [ ]:
loaded_model = tf.keras.models.load_model(filename)
mod_t1_predict = np.argmax(model_t1.predict(X_test), axis=1) 
loaded_t1_predict = np.argmax(loaded_model.predict(X_test), axis=1)

# Check the difference

print(f'Difference in predictions: Saved model vs. original model is {np.sum(loaded_t1_predict - mod_t1_predict)}')

In [ ]:
plot_summary_results(model_t1, history_t2, 20)

In [ ]:
history_t3 = model_t1.fit(datagen.flow(X_train, y_train, batch_size=32), 
                          steps_per_epoch=int(len(X_train)/32), epochs=10, 
                          validation_data=(X_test, y_test))